In [20]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rescale=1.0/65336.0,
    samplewise_center=False,
    samplewise_std_normalization=False,
    featurewise_center=True,
    featurewise_std_normalization=True,
    horizontal_flip=False)

print('Fitting datagen...')
# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(X_train[0])
print('Fitted')

X_train_preproc = [X_train[0].copy(), X_train[1].copy()]
X_val_preproc = [X_val[0].copy(), X_val[1].copy()]

X_train_preproc[0] = datagen.standardize(X_train_preproc[0].astype(float))
X_val_preproc[0] = datagen.standardize(X_val_preproc[0].astype(float))

# preprocessors = {
#     'psfs': datagen,
#     'coords': coords_scaler
# }

# import pickle
# with open('./scalers.p', 'wb') as f:
#     pickle.dump(preprocessors, f)

Fitting datagen...
Fitted


In [26]:
 # Vision transformer training

from tensorflow import keras
from tensorflow.keras import layers, Sequential
from tensorflow.keras import optimizers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tqdm.keras import TqdmCallback
from tensorflow.keras import regularizers
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, LayerNormalization
from tensorflow.keras.models import Model
from vit_keras import vit

# Assuming your input images have size (image_size, image_size, num_channels)
image_size = X_train_preproc[0].shape[1]
num_channels = X_train_preproc[0].shape[-1]
num_classes = 1  # Regression task, predicting a single continuous value

# Create the Vision Transformer model using the vit_keras library
inputs = Input(shape=(image_size, image_size, num_channels))


print(inputs.shape)
vit_model = vit.vit_b16(image_size=image_size, 
                        activation='sigmoid',
                        pretrained=True,
                        include_top=False,
                        pretrained_top=False)

x = vit_model(inputs)
# Add additional layers for regression prediction
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
regression_output = Dense(num_classes, activation='linear')(x)  # Linear activation for regression

# Combine the Vision Transformer backbone with the regression head
model = Model(inputs=inputs, outputs=regression_output)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])


batch_size = 64
epochs = 5000
lr = 0.0001

# # Model refining
# model = keras.models.load_model('./latest_vit_model/')
    
# n_layers = len(model.layers)
# for i in range(0, len(model.layers)-4):
#     model.layers[i].trainable = False
# assert model.trainable == True

    
# # Print a summary of the model architecture
# model.summary()


model.compile(loss='mean_squared_error', optimizer=optimizers.AdamW(learning_rate=lr), metrics=['mean_absolute_error'])

callbacks = [
    ReduceLROnPlateau(monitor='val_mean_absolute_error', factor=0.1,
                      patience=50, verbose=True, mode='min', min_delta=5, min_lr=1e-6,),
    EarlyStopping(monitor='val_mean_absolute_error', patience=75,
                  verbose=False, min_delta=1, restore_best_weights=True),
    TqdmCallback(verbose=1),
]


history = model.fit(X_train_preproc, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val_preproc, y_val), callbacks=callbacks, shuffle=True, verbose=True)


/home/miguel/Projects/uni/venv/lib/python3.8/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


(None, 128, 128, 3)


/home/miguel/Projects/uni/venv/lib/python3.8/site-packages/vit_keras/utils.py:81: UserWarning: Resizing position embeddings from 24, 24 to 8, 8
  warnings.warn(


2023-09-12 10:07:53.838912: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 13678018560 exceeds 10% of free system memory.
2023-09-12 10:07:59.192267: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 13678018560 exceeds 10% of free system memory.


Epoch 1/5000


...

1088/1088 [==============================] - 223s 205ms/step - loss: 77415.6875 - mean_absolute_error: 162.0187 - val_loss: 2929.1392 - val_mean_absolute_error: 29.9058 - lr: 1.0000e-05
Epoch 94/5000
 640/1088 [================>.............] - ETA: 1:30 - loss: 76840.4219 - mean_absolute_error: 161.7872

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1088/1088 [==============================] - 224s 206ms/step - loss: 77288.8594 - mean_absolute_error: 162.0219 - val_loss: 2934.6609 - val_mean_absolute_error: 30.2624 - lr: 1.0000e-05
Epoch 100/5000
  68/1088 [>.............................] - ETA: 3:26 - loss: 72779.7734 - mean_absolute_error: 157.6171

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1088/1088 [==============================] - 224s 206ms/step - loss: 77269.5859 - mean_absolute_error: 161.8777 - val_loss: 2991.6426 - val_mean_absolute_error: 28.4231 - lr: 1.0000e-05

